In [ ]:
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [ ]:
"""
    @input:
        y_test (array-like): True target values.
        custom_pred (DataFrame): DataFrame with 'lower' and 'upper' columns representing the prediction intervals.
    @output:
        None
    @purpose:
        Prints the proportion of the prediction intervals that contain the true target values.
"""
import numpy as np

def get_interval_accuracy(holdout_y, pred_y):
    custom_correct = np.mean((pred_y['lower'] < holdout_y) & (holdout_y < pred_y['upper']))
    print(f"{custom_correct:.2%} of the prediction intervals contain true target.")


In [ ]:
# Define a function to apply exponent to each cell of the DataFrame
def exp_func(x):
    return np.exp(x)

In [ ]:
 """
    @input:
        pred_obj (numpy array): A NumPy array with the point estimate, lower bound,
            and upper bound predictions.
        actual (pandas Series): A Series containing the actual values of the target variable.
        level (float): The level of the prediction intervals.

    @outpu:
        dict: A dictionary containing a summary of the interval score results and
        the indices of any missing predictions.

    @purpose: Calculate the interval score and coverage rate for a set of prediction intervals.

    @references:
        This function is based on the `interval_score` function in the stat447 ozone_quantregRF.pdf
        course notes from the University of British Columbia, available at:
        https://uglab.stat.ubc.ca/~h.joe/stat447-2023/Examples/ozone-quantregRF.pdf
    """



def interval_score(pred_obj, actual, level):
    alpha = 1 - level
    ilow = actual < pred_obj[:, 1]  # overestimation
    ihigh = actual > pred_obj[:, 2] # underestimation
    sum_length = np.sum(pred_obj[:, 2] - pred_obj[:, 1]) # sum of lengths of prediction intervals
    sum_low = np.sum(pred_obj[ilow, 1] - actual[ilow]) * 2 / alpha
    sum_high = np.sum(actual[ihigh] - pred_obj[ihigh, 2]) * 2 / alpha
    avg_length = sum_length / pred_obj.shape[0]
    IS = (sum_length + sum_low + sum_high) / pred_obj.shape[0]  # average length + average under/over penalties
    cover = np.mean((actual >= pred_obj[:, 1]) & (actual <= pred_obj[:, 2]))
    summ = [level, avg_length, IS, cover] 
    # summary with level, average length, interval score, coverage rate
    imiss = np.where(ilow | ihigh)[0]
    return {'summary': summ, 'imiss': imiss}

In [ ]:
# A function to calculate prediction interval
def interval_score_linear(prediction, actual, lower, upper, alpha):
    ilow = actual < lower  # overestimation
    ihigh = actual > upper # underestimation
    sum_length = np.sum(upper - lower) # sum of lengths of prediction intervals
    sum_low = np.sum(prediction[ilow] - actual[ilow]) * 2 / alpha
    sum_high = np.sum(actual[ihigh] - prediction[ihigh]) * 2 / alpha
    avg_length = sum_length / prediction.shape[0]
    return (sum_length + sum_low + sum_high) / prediction.shape[0]  # average length + average under/over penalties

In [ ]:
def AIC_linear(model, X, y):
    y_pred = model.predict(X)
    resid = y - y_pred
    rss = np.sum(resid**2)
    n = len(y)
    num_features = model.df_model
    
    aic = 2*num_features + n*np.log(rss/n)
    
    return aic

In [ ]:
def AIC_gb(model, X, y):
    # Make predictions on the training data
    y_pred = model.predict(X)
    # Calculate the sum of squared errors (SSE)
    sse = ((y - y_pred) ** 2).sum()
    # Calculate the number of parameters in the model
    n_params = model.n_estimators * (2 + 2*model.max_depth)
    # Calculate the in-sample AIC
    n_obs = len(y)
    aic = n_obs * np.log(sse/n_obs) + 2 * n_params
    return aic

In [ ]:
def AIC_rf(model, X, y):
    # Make sure the model is fitted with the 'oob_score' parameter set to True
    assert model.oob_score, "Model must be fitted with 'oob_score=True'."

    # Get the OOB error
    oob_error = 1 - model.oob_score_

    # Calculate the sum of squared errors (SSE)
    sse = ((y - oob_error) ** 2).sum()

    # Estimate the number of parameters in the model
    # Here we use the total number of nodes in the forest as a proxy for the number of parameters
    n_params = sum([tree.tree_.node_count for tree in model.estimators_])

    # Calculate the AIC
    n_obs = len(y)
    aic = n_obs * np.log(sse/n_obs) + 2 * n_params

    return aic

In [ ]:
def plot_prediction_interval(actual, lower_bound, upper_bound, quantile_level):
    # Exponentiate the target variable and predictions
    actual_exp = np.exp(actual)
    lower_bound_exp = np.exp(lower_bound)
    upper_bound_exp = np.exp(upper_bound)

    # Reset the index of actual_exp to match the index of lower_bound_exp and upper_bound_exp
    actual_exp = actual_exp.reset_index(drop=True)

    # Create a boolean mask indicating whether the actual values are within the prediction interval
    in_interval = (actual_exp >= lower_bound_exp) & (actual_exp <= upper_bound_exp)

    # Plot the prediction interval
    plt.figure(figsize=(15, 6))
    plt.fill_between(np.arange(len(actual_exp)), lower_bound_exp, upper_bound_exp, alpha=1, color='cornflowerblue', label='Prediction interval')
    plt.scatter(np.arange(len(actual_exp)), actual_exp, color='lime', alpha=1, label='not covered')
    plt.scatter(np.arange(len(actual_exp))[in_interval], actual_exp[in_interval], color='orangered', alpha=1, label='Covered')
    plt.title(f'Prediction interval plot for {quantile_level}% quantile level')
    plt.xlabel('Observation index')
    plt.ylabel('Price')
    plt.legend()
    plt.show()